In [19]:
import cv2
import numpy as np
from tqdm import tqdm
import pandas as pd

In [ ]:
video_path = R"..\..\res\pont-cour.mp4"
pixels_to_meters = 17 / 1000
fps = 30
delta_t = 1 / fps
frame_skip = 1
max_frames = 100

stabilize = np.array([
    [1500, 900],
    [1300, 600],
    [400, 600],
    [250, 900]
], dtype=np.int32)

In [21]:
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
frame_height, frame_width = frame.shape[:2]

In [22]:
mask = np.zeros((frame_height, frame_width), dtype=np.uint8)
cv2.fillPoly(mask, [stabilize], 1)
prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

v_list = []
frame_idx = 0

temps = []
v_mean = []
v_std = []

In [23]:
with tqdm() as pbar:
    while True:
        for _ in range(frame_skip):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None,
                                            0.5, 3, 15, 3, 5, 1.2, 0)

        vx, vy = flow[..., 0], flow[..., 1]
        v = np.sqrt(vx**2 + vy**2)

        v_masked = v * mask
        v_filtered = v_masked[(v_masked > 0.5) & (v_masked < 20)]

        if v_filtered.size > 0:
            mean_pix = np.mean(v_filtered)
            std_pix = np.std(v_filtered)

            vitesse = mean_pix * pixels_to_meters / delta_t
            ecart_type = std_pix * pixels_to_meters / delta_t

            temps.append(frame_idx * delta_t)
            v_mean.append(vitesse)
            v_std.append(ecart_type)

        prev_gray = gray.copy()
        frame_idx += 1
        if max_frames and frame_idx >= max_frames:
            break
        pbar.update(1)

cap.release()

99it [01:00,  1.62it/s]


In [24]:
df = pd.DataFrame({
    "time_s": temps,
    "vitesse_surface_m_s": v_mean,
    "ecart_type": v_std
})

df.to_csv(R".\artifact\data.csv", index=False)